In [8]:
import wsi_preprocessing as pp
import os
import json
import shutil
import pandas as pd
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm
from torchvision.models import ResNet50_Weights
import random

In [9]:
def create_average_embedding(base_dir, case_id, slide_name):
    case_dir = os.path.join(base_dir, case_id)
    biospecimen_dir = os.path.join(case_dir, "Biospecimen")
    tiles_dir = os.path.join(biospecimen_dir, "Tiles")
    metadata_path = os.path.join(case_dir, "aggregated_data", f'{case_id}_data.json')

    tile_csv_path = None
    
    for file in os.listdir(tiles_dir):
        if file.endswith('filtered_tiles.csv') and slide_name in file:
            tile_csv_path = os.path.join(tiles_dir, file)
            break

    if not tile_csv_path:
        print(f"No valid tile CSV found for slide {slide_name} in case {case_id}.")
        return
    
    # randomly select a subset of tiles for faster processing
    tile_df = pd.read_csv(tile_csv_path)
    tile_paths = tile_df['tile_path'].dropna().tolist()
    tile_paths = random.sample(tile_paths, min(200, len(tile_paths)))

    if len(tile_paths) == 0:
        print(f"No tiles found in {tile_csv_path}.")
        return
    
    # Load pre-trained ResNet model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)
    resnet.fc = nn.Identity()  # Remove classification head to extract embeddings
    resnet = resnet.to(device)
    resnet.eval()

    # Image preprocessing pipeline
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Calculate embeddings for all tiles
    embeddings = []
    for tile_path in tqdm(tile_paths, desc=f"Processing tiles for slide {slide_name}", unit="tile"):
        try:
            tile_full_path = os.path.join(tiles_dir, tile_path)  # Full path to tile
            image = Image.open(tile_full_path).convert('RGB')
            input_tensor = preprocess(image).unsqueeze(0).to(device)  # Add batch dimension
            
            with torch.no_grad():
                embedding = resnet(input_tensor).squeeze(0)  # Remove batch dimension
            embeddings.append(embedding)
        except Exception as e:
            print(f"Error processing tile {tile_path} in case {case_id}: {e}")
    
    # Compute the average embedding
    if len(embeddings) > 0:
        avg_embedding = torch.stack(embeddings).mean(dim=0).cpu().tolist()
    else:
        print(f"No embeddings generated for slide {slide_name} in case {case_id}.")
        return
    
    print(f"Average embedding for slide {slide_name} in case {case_id}: {avg_embedding}")
    return avg_embedding

def create_WSI_embeddings(base_dir="cases"):
    case_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

    for case_id in case_dirs:
        case_dir = os.path.join(base_dir, case_id)
        biospecimen_dir = os.path.join(case_dir, "Biospecimen")
        metadata_path = os.path.join(case_dir, "aggregated_data", f'{case_id}_data.json')

        if not os.path.exists(metadata_path):
            print(f"No metadata found for case {case_id}, skipping...")
            continue
        
        with open(metadata_path, 'r') as f:
            case_metadata = json.load(f)

        slide_names_to_process = []
        
        biospecimen_data = case_metadata.get('biospecimen', {}).get('biospecimen_data', [])

        # Collect svs image file names from JSON metadata file
        for sample in biospecimen_data:
            if sample['sample_type'] == "Primary Tumor":
                for slide in sample.get('slides', []):
                    image_name = slide['image_file_name']
                    embedding = create_average_embedding(base_dir, case_id, image_name)
                    slide['embedding'] = embedding
        
        with open(metadata_path, 'w') as f:
            json.dump(case_metadata, f, indent=4)
        print(f"Embeddings added for case {case_id} in the metadata.")

In [10]:
def preprocess_WSI_slides(base_dir="cases", hard_reset=False):
    case_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    general_metadata_dir = os.path.join(base_dir, "GENERAL_METADATA")
    os.makedirs(general_metadata_dir, exist_ok=True)
    
    if "GENERAL_METADATA" in case_dirs:
        case_dirs.remove("GENERAL_METADATA")

    for case_id in case_dirs:
        case_dir = os.path.join(base_dir, case_id)
        biospecimen_dir = os.path.join(case_dir, "Biospecimen")
        metadata_path = os.path.join(case_dir, "aggregated_data", f'{case_id}_data.json')

        if not os.path.exists(metadata_path):
            print(f"No metadata found for case {case_id}, skipping...")
            continue
        
        with open(metadata_path, 'r') as f:
            case_metadata = json.load(f)
        
        # Check for Tiles.part
        tiles_part_dir = os.path.join(biospecimen_dir, 'Tiles.part')
        if os.path.exists(tiles_part_dir):
            shutil.rmtree(tiles_part_dir)
            print(f"Partially downloaded directory {tiles_part_dir} and its contents removed successfully.")

        # If Tiles exists, skip
        tiles_dir = os.path.join(biospecimen_dir, 'Tiles')
        if os.path.exists(tiles_dir):
            if hard_reset:
                shutil.rmtree(tiles_dir)
            else:
                print(f"Skipping case {case_id} because 'Tiles' directory already exists.")
                continue

        slides_to_process = []
        slide_names_to_process = []
        
        biospecimen_data = case_metadata.get('biospecimen', {}).get('biospecimen_data', [])
        
        # Collect svs image file names from JSON metadata file
        for sample in biospecimen_data:
            if sample['sample_type'] == "Primary Tumor":
                for slide in sample.get('slides', []):
                    image_name = slide['image_file_name']
                    slide_path = os.path.join(biospecimen_dir, image_name)
                    if os.path.exists(slide_path):
                        slides_to_process.append(slide_path)
                        slide_names_to_process.append(image_name)
                        for file in os.listdir(os.getcwd()):
                            # check for garbage files
                            if image_name in file:
                                file_path = os.path.join(os.getcwd(), file)
                                if os.path.isdir(file_path):
                                    shutil.rmtree(file_path, ignore_errors=True)
                                    print(f"Removed directory: {file_path}")
                                elif os.path.isfile(file_path):
                                    os.remove(file_path)
                                    print(f"Removed file: {file_path}")
                    else:
                        print(f"Slide {image_name} not found for case {case_id}")
        
        if not slides_to_process:
            print(f"No Primary Tumor slides found for case {case_id}")
            continue
        
        output_folder = os.path.join(biospecimen_dir)
        os.makedirs(output_folder, exist_ok=True)

        slide_csv_path = os.path.join(output_folder, 'slides_mpp_otsu.csv')
        consolidated_csv_path = os.path.join(output_folder, 'consolidated.csv')
        tiles_filter_path = os.path.join(output_folder, 'tiles_filter.csv')
        filtered_tiles_path = os.path.join(output_folder, 'filtered_tiles.csv')
        
        # Commence tile cutting for current case
        pp.save_slides_mpp_otsu(slides_to_process, slide_csv_path)
        
        try:
            pp.run_tiling(
                slide_csv=slide_csv_path,
                consolidated_csv=consolidated_csv_path
            )
        except Exception as e:
            print(f"Exception during tiling for case {case_id}: {e}")
        
        pp.calculate_filters(
            slide_csv_path,
            "",
            tiles_filter_path
        )

        os.makedirs(tiles_part_dir, exist_ok=True)

        # Move generated files to appropriate folders
        for file in os.listdir(os.getcwd()):
            if "TCGA" in file and ".svs" in file:
              old_path = os.path.join(os.getcwd(), file)
              new_path = os.path.join(tiles_part_dir)
              
              # Print paths for debugging
              print(f"Attempting to move: {old_path} to {new_path}")
              
              try:
                  shutil.move(old_path, new_path)
                  print(f"Moved {file} successfully.")
              except Exception as e:
                  print(f"Failed to move {file}: {e}")

        if os.path.exists(tiles_part_dir):
            os.rename(tiles_part_dir, tiles_dir)
            print(f"Renamed {tiles_part_dir} to {tiles_dir}")

        for slide_name in slide_names_to_process:
            filters_csv_path = None
            for file in os.listdir(tiles_dir):
                if file.endswith("filters_cleanup.csv") and slide_name in file:
                    filters_csv_path = os.path.join(tiles_dir, file)
                    break

            if not filters_csv_path:
                print(f"No filters_cleanup.csv found for slide {slide_name} in case {case_id}.")
                continue

            df = pd.read_csv(filters_csv_path)
            filtered_df = df[df['bg_otsu'].notnull()]
            filtered_tiles_path = filters_csv_path.replace("filters_cleanup.csv", "filtered_tiles.csv")
            filtered_df.to_csv(filtered_tiles_path, index=False)

        print(f"Processing complete for case {case_id}")

In [11]:
if __name__=='__main__':
  preprocess_WSI_slides("cases_TEST_TRAIN_100", False)
  create_WSI_embeddings("cases_TEST_TRAIN_100")                     

Skipping case 13eff2e5-e33a-485f-9ba4-8a7ccb3c7528 because 'Tiles' directory already exists.
Skipping case 401a37f3-6630-4400-a811-a262351e37de because 'Tiles' directory already exists.
Skipping case 4f786107-3cf5-4ab3-bba4-f399dee23f0e because 'Tiles' directory already exists.
No Primary Tumor slides found for case 65bb7520-f055-43a8-b735-1152fa2c9e04
Skipping case 733d8b6a-ca9d-4a69-8c9c-1f88733e8b68 because 'Tiles' directory already exists.
Skipping case 93cd5d07-e0d3-40f8-ae7f-beadb1efb7c1 because 'Tiles' directory already exists.
Skipping case ad440651-a2de-4bb1-90da-1e5e8975ab59 because 'Tiles' directory already exists.
Skipping case c1e141af-bdd1-4176-8c0e-cf1ab0c1d9d3 because 'Tiles' directory already exists.
[1/4] cases_TEST_TRAIN_100\d4e1472a-7b5f-46f4-acb7-b961f634e28e\Biospecimen\TCGA-AA-A024-01A-01-BS1.274e4819-7d73-429b-be76-7956b4cdd48f.svs
- finding slide resolution
- calculating otsu threshold
[2/4] cases_TEST_TRAIN_100\d4e1472a-7b5f-46f4-acb7-b961f634e28e\Biospecimen\

c:\Users\davet\AppData\Local\Programs\Python\Python312\Lib\site-packages\wsi_tile_cleanup\filters\otsu.py:46: RuntimeWarning: invalid value encountered in divide
  mean2 = np.divide(a2, w2[::-1], out=np.zeros_like(a2), where=w2 != 0)[::-1]


slides info (mpp, otsu_thres) saved to cases_TEST_TRAIN_100\e394e9ec-7288-4ede-9ef8-41b631d100dd\Biospecimen\slides_mpp_otsu.csv
[1/2] cases_TEST_TRAIN_100\e394e9ec-7288-4ede-9ef8-41b631d100dd\Biospecimen\TCGA-AA-3715-01A-01-BS1.3927954b-ba83-41d4-aab5-ff73273c68a1.svs
- slide will NOT be rescaled | 0.5015 is within range of targeted mpp
- time elapsed: 0:00:26.060879
- directory TCGA-AA-3715-01A-01-BS1.3927954b-ba83-41d4-aab5-ff73273c68a1.svs_mpp-0.5_crop0-7_files/ has 11375 .png tiles
- tile_path data saved to ./TCGA-AA-3715-01A-01-BS1.3927954b-ba83-41d4-aab5-ff73273c68a1.svs_mpp-0.5_crop0-7_files-tile_path.csv
[2/2] cases_TEST_TRAIN_100\e394e9ec-7288-4ede-9ef8-41b631d100dd\Biospecimen\TCGA-AA-3715-01A-01-TS1.7d72472b-57ac-440f-92af-214864f84e1e.svs
- slide will NOT be rescaled | 0.5015 is within range of targeted mpp
- time elapsed: 0:00:18.160426
- directory TCGA-AA-3715-01A-01-TS1.7d72472b-57ac-440f-92af-214864f84e1e.svs_mpp-0.5_crop32-50_files/ has 8544 .png tiles
- tile_path dat

Processing tiles for slide TCGA-AD-6888-01A-01-TS1.e7e6ffce-c563-44ed-9483-ef7c37418b06.svs: 100%|██████████| 200/200 [00:16<00:00, 12.39tile/s]


Average embedding for slide TCGA-AD-6888-01A-01-TS1.e7e6ffce-c563-44ed-9483-ef7c37418b06.svs in case 13eff2e5-e33a-485f-9ba4-8a7ccb3c7528: [0.005945178680121899, 0.2151700258255005, 0.009287348948419094, 0.0004040766507387161, 0.004649332724511623, 0.046600695699453354, 2.02315092086792, 0.011989406310021877, 0.038611702620983124, 0.02420670911669731, 0.04581834375858307, 0.17956657707691193, 0.020823052152991295, 0.002374064177274704, 0.04776059091091156, 0.012823725119233131, 0.19088603556156158, 0.027701346203684807, 0.005407309625297785, 0.005388743709772825, 0.05844029039144516, 0.019037192687392235, 0.13606993854045868, 0.5675941705703735, 0.017848758026957512, 0.002023258013650775, 0.004635628778487444, 0.013187327422201633, 0.010286396369338036, 0.04092494398355484, 0.020525537431240082, 0.13397973775863647, 0.005414523649960756, 0.004051737487316132, 0.049125973135232925, 0.0004208772734273225, 0.051702894270420074, 0.10419812798500061, 0.06455883383750916, 0.03828766942024231

...

Average embedding for slide TCGA-AA-A024-01A-01-MS1.ad919832-1db3-4106-bc71-201480fdcb96.svs in case d4e1472a-7b5f-46f4-acb7-b961f634e28e: [0.010442817583680153, 0.23899787664413452, 0.004533492960035801, 0.0002137344126822427, 0.007859669625759125, 0.03033570945262909, 0.8728989958763123, 0.01143435575067997, 0.016726722940802574, 0.030819177627563477, 0.024081053212285042, 0.18970346450805664, 0.013444202952086926, 0.013604297302663326, 0.013958432711660862, 0.016989383846521378, 0.1846734583377838, 0.03398727998137474, 0.00932607427239418, 0.003957271110266447, 0.03761693835258484, 0.020179508253932, 0.21635621786117554, 0.40973949432373047, 0.04657357558608055, 0.0011066290317103267, 0.0034897243604063988, 0.012625731527805328, 0.014212374575436115, 0.1305263638496399, 0.0630832314491272, 0.034338369965553284, 0.0019350425573065877, 0.0045679667964577675, 0.012876603752374649, 0.004511966835707426, 0.15897081792354584, 0.12109193205833435, 0.1159447580575943, 0.03316088393330574, 0

Processing tiles for slide TCGA-AA-3715-01A-01-BS1.3927954b-ba83-41d4-aab5-ff73273c68a1.svs: 100%|██████████| 200/200 [00:14<00:00, 13.67tile/s]


Average embedding for slide TCGA-AA-3715-01A-01-BS1.3927954b-ba83-41d4-aab5-ff73273c68a1.svs in case e394e9ec-7288-4ede-9ef8-41b631d100dd: [0.002268174197524786, 0.1963742971420288, 0.004285827744752169, 0.0003896226698998362, 0.0023682217579334974, 0.018302874639630318, 1.4450651407241821, 0.007602017372846603, 0.009407070465385914, 0.014843190088868141, 0.01362099964171648, 0.1747569888830185, 0.015803096815943718, 0.032438524067401886, 0.07015013694763184, 0.0010329276556149125, 0.20263683795928955, 0.015316298231482506, 0.006648985203355551, 0.003217831952497363, 0.01996755041182041, 0.018518630415201187, 0.13145531713962555, 0.06716714799404144, 0.0027909381315112114, 0.0005466523580253124, 0.005088684149086475, 0.007421144284307957, 0.012439790181815624, 0.016481786966323853, 0.028336506336927414, 0.007685425225645304, 0.002102859551087022, 0.0008348976261913776, 0.0068718367256224155, 2.7682026484399103e-05, 0.04896717518568039, 0.04515378922224045, 0.09955650568008423, 0.023115

Processing tiles for slide TCGA-AA-3715-01A-01-TS1.7d72472b-57ac-440f-92af-214864f84e1e.svs: 100%|██████████| 200/200 [00:15<00:00, 13.27tile/s]

Average embedding for slide TCGA-AA-3715-01A-01-TS1.7d72472b-57ac-440f-92af-214864f84e1e.svs in case e394e9ec-7288-4ede-9ef8-41b631d100dd: [0.0026413716841489077, 0.2169855237007141, 0.003905486548319459, 0.00044207918108440936, 0.0025874364655464888, 0.04731054604053497, 1.294248342514038, 0.005814680363982916, 0.008724444545805454, 0.007887848652899265, 0.0149643924087286, 0.12718768417835236, 0.01915419101715088, 0.02275790274143219, 0.11298899352550507, 0.0007138210348784924, 0.21757423877716064, 0.007001897785812616, 0.00535739166662097, 0.0031958711333572865, 0.02312385104596615, 0.008879394270479679, 0.17483273148536682, 0.14127685129642487, 0.006079637445509434, 0.00045766597031615674, 0.0027073146775364876, 0.008061185479164124, 0.009513741359114647, 0.03115205466747284, 0.046598583459854126, 0.013115320354700089, 0.0018923906609416008, 0.0038143224082887173, 0.012874691747128963, 0.0015405539888888597, 0.0343417152762413, 0.10305066406726837, 0.07410979270935059, 0.0404546596